In [15]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup

In [7]:
# Main one


# Set options for Microsoft Edge
options = Options()
options.use_chromium = True  # Use this option if using Edge Chromium

# Specify the path to the WebDriver executable
webdriver_path = r"C:\Users\anmol.chopra\Downloads\edgedriver_win64 (1)\msedgedriver.exe"

# Initialize the WebDriver
service = Service(webdriver_path)
driver = webdriver.Edge(service=service, options=options)

# Open a particular website
url = 'https://www.cmegroup.com/markets/agriculture/oilseeds/soybean.volume.html'
driver.get(url)

html_source = driver.page_source

soup = BeautifulSoup(html_source, 'html.parser')

# Locate all tables
tables = soup.find_all('div', {'class': 'main-table-wrapper'})

# Check if there are enough tables
if len(tables) > 1:
    table = tables[1].find('table')  # Select the second table
else:
    raise ValueError("The second table is not found on the page.")


label_element = driver.find_element(By.XPATH, '//label[text()="Trade Date"]')

# Find the adjacent sibling element containing the date
date_element = label_element.find_element(By.XPATH, './following-sibling::div//span[@class="button-text"]')

# Extract the date text from the element
date_text = date_element.text

# Extract table headers
headers = []
header_rows = table.find_all('thead')
for header_row in header_rows:
    cols = header_row.find_all('th')
    for col in cols:
        colspan = int(col.get('colspan', 1))
        headers.extend([col.get_text(separator=' ').strip()] * colspan)

# Extract table rows
rows = []
for row in table.find_all('tbody')[0].find_all('tr'):
    cols = row.find_all('td')
    if len(cols) > 0:
        cols = [ele.get_text().strip() for ele in cols]
        rows.append(cols)

# Create DataFrame
df = pd.DataFrame(rows, columns=headers[:len(rows[0])])

# Print DataFrame
print(df)

driver.quit()


       Month   Volume Volume Volume   Volume Volume Volume Volume Volume  \
0   AUG 2024   35,320      0  2,298   37,618      0  2,056    242  1,083   
1   SEP 2024   45,880      0     16   45,896      0     16      0    476   
2   NOV 2024  181,917      0  1,451  183,368    232  1,218      1  3,630   
3   JAN 2025   33,168      0      0   33,168      0      0      0    661   
4   MAR 2025   25,824      0  5,688   31,512      0  5,688      0    325   
5   MAY 2025   12,340      0    478   12,818      0    478      0    406   
6   JUL 2025    6,102      0      0    6,102      0      0      0     16   
7   AUG 2025      210      0      0      210      0      0      0      0   
8   SEP 2025      140      0      0      140      0      0      0      0   
9   NOV 2025    2,151      0      0    2,151      0      0      0      0   
10  JAN 2026       82      0      0       82      0      0      0      0   
11  MAR 2026       22      0      0       22      0      0      0      0   

   Deliveri

In [8]:
df = pd.DataFrame(rows, columns=headers[:len(rows[0])])
df = df[['Month','Open Interest']]
df.columns = ['Month','OI','Open Interest']
df = df.drop(columns='OI')
df

,Month,Open Interest
0,AUG 2024,"-17,755"
1,SEP 2024,+867
2,NOV 2024,"-1,480"
3,JAN 2025,"+4,266"
4,MAR 2025,"+2,573"
5,MAY 2025,"+2,082"
6,JUL 2025,+866
7,AUG 2025,+5
8,SEP 2025,+56
9,NOV 2025,-60


In [9]:
df_new = df.T
df_new = df_new.astype(str)
df_new.columns = df_new.iloc[0]
df_new = df_new.drop(df_new.index[0])
df_new.insert(0,'Date',date_text.split(', ')[1] )
# df_new.to_excel('OI Data.xlsx',index=False)
df_new.to_csv('OI Data.csv',index=False)

df_new

Month,Date,AUG 2024,SEP 2024,NOV 2024,JAN 2025,MAR 2025,MAY 2025,JUL 2025,AUG 2025,SEP 2025,NOV 2025,JAN 2026,MAR 2026
Open Interest,29 Jul 2024,"-17,755",+867,"-1,480","+4,266","+2,573","+2,082",+866,+5,+56,-60,+59,+4


In [8]:
import pandas as pd
from openpyxl import Workbook


df1 = pd.read_csv(r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\code\OI\OI Data-Corn.csv')
df2 = pd.read_csv(r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\code\OI\OI Data-HRW.csv')
df3 = pd.read_csv(r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\code\OI\OI Data-Soybean.csv')
df4 = pd.read_csv(r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\code\OI\OI Data-Soymeal.csv')
df5 = pd.read_csv(r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\code\OI\OI Data-Soyoil.csv')
df6 = pd.read_csv(r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\code\OI\OI Data-SRW.csv')

file_path = r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\code\OI\OI Data.xlsx'

wb = Workbook()

wb.save(file_path)

# Save dataframes to different sheets in the cleared Excel file
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    df1.to_excel(writer, sheet_name='Corn', index=False)
    df2.to_excel(writer, sheet_name='HRW', index=False)
    df3.to_excel(writer, sheet_name='Soybean', index=False)
    df4.to_excel(writer, sheet_name='Soymeal', index=False)
    df5.to_excel(writer, sheet_name='Soyoil', index=False)
    df6.to_excel(writer, sheet_name='SRW', index=False)


In [34]:
# this is for 3 products and extracting barchart
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

# Set options for Microsoft Edge
options = Options()
options.use_chromium = True  # Use this option if using Edge Chromium

# Specify the path to the WebDriver executable
webdriver_path = r"C:\Users\anmol.chopra\Downloads\edgedriver_win64 (1)\msedgedriver.exe"

# Initialize the WebDriver
service = Service(webdriver_path)
driver = webdriver.Edge(service=service, options=options)

# Open a particular website
url = 'https://www.barchart.com/futures/quotes/HE*0/futures-prices'
driver.get(url)
# Increase the timeout duration to handle potential delays
timeout = 20

try:
    # Wait for the table wrapper to be present in the DOM and visible
    table_element = WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'bc-table-wrapper'))
    )
    # Wait for the table to be interactable
    WebDriverWait(driver, timeout).until(
        EC.visibility_of(table_element)
    )
    
    # Get the HTML content of the table
    table_html = table_element.get_attribute('outerHTML')

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(table_html, 'html.parser')
    table = soup.find('bc-data-grid', {'id': 'table-guid-ea68b6f9-e2c7-114f-d7c2-0fa52e344b86'})

    if table:
        # Extract data from the table
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all('td')
            cell_values = [cell.text.strip() for cell in cells]
            print(cell_values)
    else:
        print("Table not found")
except TimeoutException:
    print("Timed out waiting for the table to load")
finally:
    # Close the WebDriver
    driver.quit()

Table not found


In [32]:
print(table)

<div class="bc-table-wrapper barchart-table-sticky-150 table-guid-52b048eb-5dd1-ba4a-e537-ee5313b6c76a symbol-details links-enabled set-min-height sticky-row table-size-12" data-ng-class="{'sticky-row': sticky &amp;&amp; content &amp;&amp; content[0].count, 'has-error': error, 'symbol-details': options.symbolDetails, 'is-news-view': options.isNewsView, 'links-enabled':options.quickLinks , 'datatable-processing': processing, 'set-min-height': options.setMinHeight, 'hidden': dataLoading}">
<bc-data-grid data-ng-attr-lazy-row-creation="[[ lazyRows ? true : undefined ]]" data-ng-attr-suspend-occluded-bindings="[[ lazyRows ? true : undefined ]]" data-ng-hide="error" externalstylestemplateid="grid-styles" fast-row-creation="false" focusentirerow="" horizontalgridlines="" id="table-guid-52b048eb-5dd1-ba4a-e537-ee5313b6c76a" lazy-row-creation="true" markaltrows="" norowreorder="" rowclassbinding="this.rowClass" style="" suspend-occluded-bindings="true" verticalgridlines="">
</bc-data-grid>
</d

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

# Find the div containing the data
div = soup.find('div', class_='bc-datatable ng-isolate-scope')

# Extract the data from the 'data-api-config' attribute
data_config = json.loads(div['data-api-config'])

# The actual data is likely loaded dynamically, so we need to make an API call
api_url = f"https://www.barchart.com/proxies/core-api/v1/quotes/get?lists={data_config['api']['lists']}&fields={data_config['api']['fields']}&root={data_config['api']['root']}"

# Make the API request
api_response = requests.get(api_url)
data = api_response.json()

# Create a DataFrame from the data
df = pd.DataFrame(data['results'])

# Display the first few rows of the DataFrame
print(df.head())

In [26]:
soup

<html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><script async="" src="https://static.kueezrtb.com/js/latest_cls.js?_=1722356188065" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-10809465205&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-PE0FK9V6VN&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script src="https://securepubads.g.doubleclick.net/pagead/managed/js/gpt/m202407250101/pubads_impl_page_level_ads.js"></script><script async="" src="https://ml314.com/tag.aspx?306"></script><script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-TKSHCJ8"></script><script async="" charset="utf-8" crossorigin="anonymous" integrity="sha384-CUWl3AwpNax4J2/ffIDxMr+sLnEFg8pkj5QqZtMtJZtrL5bS5QTkIoza4qZakWjL" src="https://